# 模板层

# 模板语法传值

模板语法可以传递的后端python数据类型

{{}}：变量相关

{%%}：逻辑相关

对于python后端所有数据类型均可传递。

```
传递函数名会自动加括号调用，但模板语法不支持给函数传额外的参数  {{ func }}
传递类名也会自动加括号调用（实例化）{{ MyClass }}

内部能够自动判断出当前的变量名是否可以加括号调用，如果可以就会自动执行，针对的是函数名和类名。


模板语法取值：
是固定格式，只能采用 '.'，既可以点字典的键也快一点序列类型的索引
```

# 模板语法之过滤器

过滤器就类似于是模板语法内置的 内置方法。

基本语法

    {{ 数据|过滤器:参数 }}
   

常见内置方法
```
1.统计长度
s = "hello"
{{ s|length }}

2.默认值，如果前面值是True就用前面的，前面bool值是False就用后面的
b = True
{{ b|default:'啥也不是' }}

3.文件大小，自动转换成人类方便阅读的数据
file_size = 21321
{{ file_size|fileseizeformat }}

4.日期格式化
current_time = datetime.datetime.now()
{{ current_time|date:'Y-m-d H:i:s' }}

5.切片
{{ l|slice:'0:4:2' }}

6.窃取字符，拿到摘要，包含三个点
{{info|truncatechars:9}}

7.移除特定的字符
{{ msg|cut:' ' }}

8.拼接
{{ l|join:'@' }}

9.转义
默认是不会将传递过来的h5语法进行转义，传什么就打印什么。
前端
hh = '<h1>哈喽</h1>'
{{ hh|safe }}
后端
from django.utils.safestring import mark_safe
res = makr_safe()

后续写全栈项目时，前端代码不一定非要在前端写，也可以在后端写好，然后传递到前端页面
```

# 标签

for else 混合使用

```
{% for foo in lll %}
    {% if forloop.first %}
        <p>第一次循环</p>
    {% elif forloop.last %}
        <p>最后一次循环</p>
    {% else %}
        <p>{{ foo }}</p>
    {% endif %}
    {% empty %}
        <p>for循环的可迭代对象内部没有元素，根本没有办法循环</p>
{% endfor %}
```

起别名

```
在with语法内就可以通过as后面的别名快速的使用到前面非常复杂获取数据的方式
{% with d.hobby.3.info as nb %}
    <p>{{ nb }}</p>
{% endwith %}
```

# 自定义过滤器、标签、inclusion_tag

**先三步走：**
1. 在应用下创建一个名字**必须**叫templatetags文件夹
2. 在该文件夹内创建**任意**名称的py文件 如 mytag.py
3. 在该py文件中**必须**先书写下面两句话
```
    from django import template
    register = template.Library()
```

详细实现

1. 自定义过滤器

    **过滤器最多只能有两个参数。**

    在py脚本中添加
    ```
    @register.filter(name='baby')
    def my_sum(v1, v2):
        return v1+v2
    ```

    使用：
    ```
    在html文件中
    {% load mytag %}
    <p>{{ n|baby:666 }}</p>
    ```

2. 自定义标签

    **参数任意多个**

    在mytag.py脚本中添加
    ```
    @register.simple_tag(name='plus')
    def plus(a, b, c, d):
        return '{0}{1}{2}{3}'.format(a, b, c, d)
    ```

    使用
    ```
    {% load mytag %}
    标签多个参数彼此之间空格隔开
    <p>{% plus '1' '2' '3' '4' %}</p>
    ```

3. 自定义inclusion_tag

    内部原理：先定义一个方法。在页面上调用该方法，并且可以传值到方法中，该方法会生成一些数据然后传递给一个html页面。之后将渲染好的结果放到调用的位置。

    ```
    定义
    @register.inclusion_tag('left_menu.html')
    def left(n):
        data = [ '第{}项'.format(i) for i in range(n)]
        return locals()

    left_menu.html文件内容
    可以写对数据的显示

    调用
    {% left 5 %}
    ```

    使用时机：当html页面某一个地方的页面需要传参数才能够动态的渲染出来，并且在多个页面上都需要使用到该局部，那么久考虑将该局部页面做成inclusion_tag形式。

# 模板的继承

一些网站上，页面整体都差不多，只是某一些局部在做变化。
```
模板的继承，自己下你选好一个想要继承的模板页面
{% extends 'home.html' %}  子页面继承模板

继承之后子页面和模板页面长的是一摸一摸，你需要在模板页面上提前规划可以被修改的区域
{% block content %}
    模板内容
{% endblock %}

子页面就可声明想要修改哪块划定的区域
{% block content %}
    子页面内容
    子页面除了可以写自己的之外,还可以继续使用模板的内容
    {{ block.super }}
{% endblock %}
```



一般情况下模板页面上应该至少右三块可以被修改的区域

- css区域
- html区域
- js区域

每一个子页面都可以右自己独有的css代码，html代码，js代码.

# 模板的导入

```
将页面的某一个局部当成模块的形式,在需要使用的地方导入即可.
{% include login.html %}
```